#**1. Preprocessing:**


##**1.1 Import thư viện:**


In [10]:
import os
import matplotlib.pyplot as plt
import cv2
import imutils
import numpy as np
from skimage import io
from google.colab.patches import cv2_imshow


##**1.2 Tiền xử lí ảnh:**


In [11]:
%cd '/content/drive/My Drive/Colab Notebooks/Capstone_Dataset/Train'
!ls

/content/drive/My Drive/Colab Notebooks/Capstone_Dataset/Train
B  C  CT  D  E


In [ ]:
ImgTest = cv2.imread('/content/drive/My Drive/Colab Notebooks/Capstone_Dataset/Train/E/IMG_3948.JPG')

####**Show ImgTest:**

In [ ]:
#plt.axis('off')
plt.figure(figsize=(8,8))
plt.imshow(cv2.cvtColor(ImgTest,cv2.COLOR_BGR2RGB))

###**1.1.1 Convert to Grayscale Image:**


In [ ]:
ImgTest = cv2.cvtColor(ImgTest,cv2.COLOR_RGB2GRAY)

In [ ]:
plt.figure(figsize=(8,8))
plt.imshow(ImgTest,cmap=plt.get_cmap('gray'))

###**1.2.2 Resize:**

+ Thông thường, **resize** một bức ảnh thường chúng ta không quan tâm đến tỉ số giữa chiều dài và chiều rộng (**aspect ratio**) của bức ảnh, điều này sẽ dẫn đến bức ảnh sau khi **resize** có thể bị méo mó.
> + Ví dụ: 
>
><img src="https://drive.google.com/uc?id=1QMhhGN-CbhaOD-4wKK0UP0qChA-j9PZs
" alt="Compare" class="center" width=500/>
>
> Đối với hình ở giữa (**Ingore Aspect ratio**) thì khi resize lại thì ảnh sẽ bị kéo dài ra hai bên, còn hình bên tay phải (**Consider Aspect ratio**) có hình dáng gần giống với ảnh gốc hơn. 


####**Resize without aspect ratio:**

In [ ]:
ImgTest_rs = cv2.resize(ImgTest,(256,256),interpolation=cv2.INTER_AREA)
#plt.axis('off')
plt.figure(figsize=(8,8))
plt.imshow(ImgTest_rs,cmap=plt.get_cmap('gray'))

####**Resize aspect ratio:**

In [ ]:
def resize_ratio(img,self_width = 256,self_height = 256,self_inter = cv2.INTER_AREA):
    # get size of image
    (h,w) = img.shape[:2]
    dW = 0
    dH = 0
    if w < h:
        img = imutils.resize(img,width=self_width,inter=self_inter)
        dH = int((img.shape[0] - self_height)/2.0)
    else:
        img = imutils.resize(img,height=self_height,inter=self_inter)
        dW = int((img.shape[1] - self_width)/2.0)

    (h,w) = img.shape[:2]
    img = img[dH:h - dH,dW:w - dW]

    return cv2.resize(img,(self_width,self_height),interpolation=self_inter)



In [ ]:
ImgTest_ratio = resize_ratio(ImgTest)

plt.figure(figsize=(8,8))
plt.imshow(ImgTest_ratio,cmap=plt.get_cmap('gray'))

###**1.2.3 Cân bằng sáng (Histogram Equalization):**

+ Trần Đin

In [ ]:
ImgE = ImgTest
equ = cv2.equalizeHist(ImgE)
ImgE = np.hstack((ImgE,equ))

plt.figure(figsize=(8,8))
plt.imshow(ImgE,cmap=plt.get_cmap('gray'))

###**1.2.4 Bộ lọc làm mờ Bilateral(Bilateral Filtering):**


In [ ]:
Img_Bilateral = cv2.bilateralFilter(ImgTest,9,75,75)

plt.figure(figsize=(8,8))
plt.imshow(Img_Bilateral,cmap=plt.get_cmap('gray'))

###**1.2.5 Sharpening Images:**

In [ ]:
kernel = np.array([[-1,-1,-1],
                   [-1, 9,-1],
                   [-1, -1,-1]])
ImgSh = cv2.filter2D(ImgTest,-1,kernel)

In [ ]:
plt.figure(figsize=(8,8))

plt.imshow(ImgSh,cmap=plt.get_cmap('gray'))

###**1.2.6 Hàm preprocessing:**


In [ ]:
def preprocessing(Image,w=256,h=256):
    # covert to grayscale
    Image = cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY)
    
    # Equalize Histogram
    Image = cv2.equalizeHist(Image)
    
    # Bilateral Filter
    Image = cv2.bilateralFilter(Image,9,75,75)
    
    # Sharpening images 
    kernel = np.array([[-1,-1,-1],
                   [-1, 9,-1],
                   [-1, -1,-1]])
    Image = cv2.filter2D(Image,-1,kernel)
    
    # Resize aspect ratio
    Image = resize_ratio(Image,w,h)
    return Image

####**Test Preprocessing:**

In [ ]:
PathImg = '/content/drive/My Drive/Colab Notebooks/Capstone_Dataset/Train/E/IMG_20200708_145837.jpg'
ImgEx = cv2.imread(PathImg)

In [ ]:
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(8,8))
ax1.imshow(cv2.cvtColor(ImgEx,cv2.COLOR_BGR2RGB))
ax1.set_title('Original Image')
ax2.imshow(preprocessing(ImgEx),cmap=plt.get_cmap('gray'))
ax2.set_title('Preprocessing Image')

##**1.3 Ghi vào file Preprocessor.py:**


In [ ]:
%cd '/content/drive/My Drive/Colab Notebooks/CS114.K21.KHTN/[Capstone Study] New_ML_FinalProject/Preprocessing'

In [ ]:
%%writefile Preprocessor.py 
import cv2
import imutils
import numpy as np

def resize_ratio(img,self_width = 256,self_height = 256,self_inter = cv2.INTER_AREA):
    # get size of image
    (h,w) = img.shape[:2]
    dW = 0
    dH = 0
    if w < h:
        img = imutils.resize(img,width=self_width,inter=self_inter)
        dH = int((img.shape[0] - self_height)/2.0)
    else:
        img = imutils.resize(img,height=self_height,inter=self_inter)
        dW = int((img.shape[1] - self_width)/2.0)

    (h,w) = img.shape[:2]
    img = img[dH:h - dH,dW:w - dW]

    return cv2.resize(img,(self_width,self_height),interpolation=self_inter)

def preprocessing(Image,w=256,h=256):
    # covert to grayscale
    Image = cv2.cvtColor(Image,cv2.COLOR_BGR2GRAY)
    
    # Equalize Histogram
    Image = cv2.equalizeHist(Image)
    
    # Bilateral Filter
    Image = cv2.bilateralFilter(Image,9,75,75)
    
    # Sharpening images 
    kernel = np.array([[-1,-1,-1],
                   [-1, 9,-1],
                   [-1, -1,-1]])
    Image = cv2.filter2D(Image,-1,kernel)
    
    # Resize aspect ratio
    Image = resize_ratio(Image,w,h)
    return Image

In [ ]:
%%writefile __init__.py
from .Preprocessor import *